In [10]:
import pandas as pd
import os
, 
from utils.models import *

## Ingest
Bring .csv into the system

In [41]:
from ingest import load_file
os.listdir('assets/data')

['dws_wages.csv',
 'usbe_students.csv',
 'ushe_students.csv',
 'ustc_students.csv']

In [42]:
load_file('assets/data/dws_wages.csv', 'dws_wages')
load_file('assets/data/usbe_students.csv', 'usbe_students')
load_file('assets/data/ustc_students.csv', 'ustc_students')
load_file('assets/data/ushe_students.csv', 'ushe_students')

## Linking Steps

Process of Record Linkage

0. Load
1. Preprocess
2. Index
3. Compare
4. Classify
5. Evaluate
6. Update (Post MPI - Adding new information, new MPI, to MPI Master Record)

### Master Person Architectures
Visualize data architecture store of MPI raw records.  NOTE: data may not be present in all database options.  Use whichever cell aligns with configured loading behavior.

**SQL** View

In [15]:
from utils.db import get_session

with get_session() as session:
    result = session.execute(
        'SELECT * FROM master_person_long LIMIT 5'
    ).fetchall()
    count_mpi = session.execute(
        'SELECT COUNT(DISTINCT(mpi)) FROM master_person_long'
    ).fetchone()
columns = ('mpi', 'field', 'value', 'score', 'guid')
pd.DataFrame(result, columns=columns)

NameError: name 'pd' is not defined

In [ ]:
print('Total MPI in system: ', count_mpi[0])

**NoSQL** View

In [1]:
from utils.db import get_mongo
import json

db = get_mongo()
count_docs = db.raw.count_documents({})
x = db.raw.find_one({})
x['_id'] = str(x['_id'])

print(json.dumps(x, indent=2))

{
  "_id": "5fc12fe81ac382de046f4db7",
  "mpi": "6066022-9429108-14143690-5027899",
  "sources": [
    {
      "guid": 270603007341519345,
      "score": 0.0,
      "fields": [
        {
          "fieldname": "ushe_student_id_pool",
          "value": 469629.0
        },
        {
          "fieldname": "last_name_pool",
          "value": "Steed"
        },
        {
          "fieldname": "first_name_pool",
          "value": "Sakayla"
        },
        {
          "fieldname": "middle_name_pool",
          "value": "Conder"
        },
        {
          "fieldname": "birth_date_pool",
          "value": "6/20/2859"
        },
        {
          "fieldname": "gender_pool",
          "value": "m"
        },
        {
          "fieldname": "ssid_pool",
          "value": "0x23570dbc0x46ae1b78"
        },
        {
          "fieldname": "ssn_pool",
          "value": 375910347
        }
      ]
    }
  ]
}


In [2]:
# Delete all records (skip if not needed)
d = db.raw.delete_many({})
d.deleted_count

10002

In [3]:
print('Total MPI in system: ', count_docs)

Total MPI in system:  10002


## Prepare Data

Prepare identity view (MPI vectors) and data view (distinct mapped columns from source)

In [1]:
source_tablename = 'ushe_students'
# source_tablename = 'dws_wages'

In [2]:
from utils.db import query_db, get_db
from mpi.prepare import create_data_view, create_identity_view

raw, subset = create_data_view(source_tablename)
dview = subset.drop_duplicates()
iview = create_identity_view(mapped_columns=dview.columns.to_list())

Check here for potential for match.  If NO matching columns exist, mpis will need to be created for the dview.

In [3]:
# Check for match availability.  If not, halt process and create MPIs
from mpi.link import is_match_available
from mpi.update import generate_mpi, write_mpi_data, gen_mpi_insert
from mpi.update import update_mpi_vector_table

if is_match_available(dview, iview):
    print('Match available.  Proceed with linking process.')
else:
    print('Match unavailable.  Generated MPIs for data view.')
    temp = generate_mpi(dview)
    write_mpi_data(gen_mpi_insert(temp))
    update_mpi_vector_table()
    
    # Recreate a view from the MPI table with valid identity data
    iview = create_identity_view(mapped_columns=dview.columns.tolist())
    

Match available.  Proceed with linking process.


In [4]:
raw.head(1)

,index,S_INST,S_YEAR,S_TERM,S_EXTRACT,S_ID,S_ID_FLAG,S_PREVIOUS_ID,S_LAST,S_FIRST,...,S_BIA,S_TERM_ATT_CR,S_TERM_EARNED_CR,S_COLLEGE,S_MAJOR,S_COLLEGE2,S_MAJOR2,S_INAME,id,guid
0,0,1858,2018,3,E,469629.0,I,187277,Steed,Sakayla,...,None,4,126,Huntsman School of Business,Deaf Ed & Elementary Ed,School of the Arts,Network Systems,SNOW,7853,270603007341519345


In [5]:
dview.head(1)

,ushe_student_id_pool,last_name_pool,first_name_pool,middle_name_pool,birth_date_pool,gender_pool,ssid_pool,ssn_pool,guid
0,469629.0,Steed,Sakayla,Conder,6/20/2859,m,0x23570dbc0x46ae1b78,375910347,270603007341519345


In [6]:
iview.head(1)

,birth_date_pool,gender_pool,middle_name_pool,last_name_pool,first_name_pool,ushe_student_id_pool,ssn_pool,ssid_pool,freq_score
0,6/20/2859,m,Conder,Steed,Sakayla,469629.0,375910347,0x23570dbc0x46ae1b78,1.0


## Building record linkage and mpi classification

In [7]:
from mpi.preprocess import clean_raw, match_dtype

### Preprocessing

Standardize data across data and identity views.

In [9]:
# Match Dtypes - Align data types prior to cleaning.
#    This helps the cleaner by segmenting string/object and numeric fields

if hasattr(iview, 'freq_score'):
    iscore = iview[['mpi', 'freq_score']]
else:
    iscore = None

# Cast columns to matching datatypes for comparisons later on
source_data, id_data = match_dtype(dview, iview)  

# Clean data and re-index comparison.
subset = clean_raw(subset)
source_data = clean_raw(source_data)
id_data = clean_raw(id_data)

KeyError: "['mpi'] not in index"

In [9]:
source_data.head(1)

,ushe_student_id_pool,last_name_pool,first_name_pool,middle_name_pool,birth_date_pool,gender_pool,ssid_pool,ssn_pool,guid
0,469629.0,steed,sakayla,conder,6/20/2859,m,0x23570dbc0x46ae1b78,375910347,270603007341519345


In [10]:
id_data.head(1)

,birth_date_pool,ushe_student_id_pool,middle_name_pool,ssid_pool,ssn_pool,gender_pool,first_name_pool,last_name_pool
0,6/20/2859,469629.0,conder,0x23570dbc0x46ae1b78,375910347,m,sakayla,steed


In [11]:
iscore.head(1)

AttributeError: 'NoneType' object has no attribute 'head'

## Indexing

Make record pairs - pair rows needing match to potential identity candidates.

Indexing serves two purposes:

1. Create the list of pairs to check (candidate link).  Example: row 1 from table 1 to row 199 from table 2.

2. Reduce the potential number of pairs to check (candidates).

In [36]:
from mpi.link import build_indexer, match_columns

In [37]:
# Create indexer on dataview
#    Indexer is a set of rules to generate 
#    candidate matches from data -> identities

source_matched, id_matched = match_columns(source_data, id_data)

indexer = build_indexer(source_matched)

# Check index algorithms (generated from data view columns)
indexer.algorithms

[<SortedNeighbourhood left_on='middle_name_pool', right_on='middle_name_pool'>,
 <SortedNeighbourhood left_on='last_name_pool', right_on='last_name_pool'>,
 <SortedNeighbourhood left_on='first_name_pool', right_on='first_name_pool'>,
 <Block left_on='ssn_pool', right_on='ssn_pool'>]

In [38]:
# Run indexer on dataview, identity view
candidates = indexer.index(source_matched, id_matched)

# Full indexing is a cross join of data and all possible identities.

# Demonstrating full indexing size:
print('Full Index Length: ', len(source_data) * len(id_data))

# Examine multi indices.  On the left is the data view index.  Right identity.
print('Algorithmic Index Length: ', len(candidates))

# Estimate Savings
print('Savings: ', (1- len(candidates)/(len(source_data) * len(id_data))) * 100)

# Preview indices:
for pair in candidates[0:5]:
    print(f'Data-row {pair[0]}', f'ID-row {pair[1]}')

Full Index Length:  100000000
Algorithmic Index Length:  57599
Savings:  99.942401
Data-row 0 ID-row 1849
Data-row 0 ID-row 6377
Data-row 0 ID-row 7005
Data-row 0 ID-row 9087
Data-row 0 ID-row 9412


## Comparing

Indexing does not normally store the outcome of its findings.  Indexing algorithms are meant to be fast, can be error prone.  Algorithms can be tuned for string (many), numeric, and time/date fields.

The output of comparison is a clean feature matrix for the classifier to train/predict on.

In [39]:
from mpi.link import build_comparator

In [40]:
# Create comparator on dataview
#    Comparator is a set of algorithms for each feature to be compared.
#    These are genearlly much more expensive compared to indexing functions
cmp = build_comparator(source_matched)

# Check comparison algorithms and fields
cmp.features

[<Numeric 'ssn_pool'>,
 <String 'middle_name_pool'>,
 <String 'last_name_pool'>,
 <String 'first_name_pool'>,
 <String 'gender_pool'>]

In [41]:
# Compute comparisons
#    Gives clean match dataset for classification
comparisons = cmp.compute(candidates, source_data, id_data)
comparisons.head()

ssn_pool  middle_name_pool  last_name_pool  first_name_pool  \
0 1849       0.0               0.0             0.0              0.0   
  6377       0.0               0.0             0.0              0.0   
  7005       0.0               0.0             0.0              0.0   
  9087       0.0               0.0             1.0              0.0   
  9412       0.0               0.0             1.0              0.0   

        gender_pool  
0 1849          1.0  
  6377          0.0  
  7005          0.0  
  9087          1.0  
  9412          0.0

## Classification

Score candidates for match.  

#### Two approaches: Supervised vs Unsupervised
 * **Supervised** approach requires a training set.
 * **Unsupervised** does not require a training set and operates on only on the comparison table itself.

In [42]:
from mpi.link import estimate_true, build_classifier

# Get estimated true linkages for supervised model
links_true = estimate_true(comparisons)

# Create classifier
clf = build_classifier('logistic', comparisons, match_index=links_true)

# Check probabilities (score) of each comparison -- NOT IN USE IN THIS VERSION
predictions = clf.prob(comparison_vectors=comparisons)
predictions

0     1849    0.000007
      6377    0.000001
      7005    0.000001
      9087    0.000271
      9412    0.000050
                ...   
9999  3443    0.000050
      3728    0.000039
      6632    0.000007
      8916    0.000189
      9871    0.000007
Length: 57599, dtype: float64

## Evaluate
Express classification quality and explore outliers

In [43]:
from recordlinkage import reduction_ratio
from recordlinkage import confusion_matrix

links_pred = clf.predict(comparison_vectors=comparisons)

rratio = reduction_ratio(links_pred, source_data)
cmatrix = confusion_matrix(links_true, links_pred, candidates)

In [44]:
# Review confusion matrix
#    TP-FN
#    |  |
#    FP-TN
print(cmatrix)

# Review reduction ratio
print(rratio)

[[  674     0]
 [    0 56925]]
0.9999865186518652


The confusion matrix may not be particularly useful here as generation of true links is prone to error. The reduction ratio is more sensitive than binary predictions in this case.

In [45]:
# Review findings
#   Interesting that the logistic predicted an MPI indices for each given an incomplete target list.

# Is the relationship 1,1?
split_list = lambda x: ([ix[0] for ix in x], [ix[1] for ix in x])
i1, i2 = split_list(links_pred)
len(list(set(i1))), len(list(set(i2)))

(674, 674)

## Update

Append matched MPIs and match score to data view and merge to original data.

In [46]:
from mpi.link import expand_match_to_raw

# Join data view (DISTINCT identities in source table), now containing matched and generated MPIs, to raw table.
#    This can be done a few ways.  Here, the data view (whose columns have been renamed and processed)
#    is joined to the original subset (whose columns were just renamed).  The subset is then indexed back 
#    unto the raw table so original column names and source formatting are preserved.


raw, matched, unmatched = expand_match_to_raw(raw, subset, source_data, id_data, links_pred)
raw.head(1)

,BIA,DISABLED,DISPLACEDHOMEMAKER,DWS,ECON,LEP,PELL,SINGLEPARENT,U_AGE,U_BIRTH_DT,...,U_START_DATE,U_STATE_ORIGIN,U_STOP_DATE,U_SUBJ,U_SUFFIX,U_TITLE,U_YEAR,id,guid,mpi
0,0,1,1,0,0,1,1,0,42,4/15/2110,...,20162189,DE,58081754,FLUID POWER HYDRAULICS,None,WASHINGTON CO. ECONOMIC TRAINING,2011,7547,930450171738989444,9197423-1362941-5290047-14787063


### Adding missing data, repeated data

When MPIs are generated for unmatched records, all data is used to populate the MPI table.  For matched MPIs, an update to missing information now needs to be performed to capture any new data.

What information is populated can have large performance and storage ramifications.
- If repeat data is kept, the initial identity view will need consolidation to reduce the number of possible indexes and comparisons.
- If repead data is NOT kept, significant information for background updates in match quality will be lost.

In [28]:
# TODO

### De-Identification

Create de-identified table while match available in memory or as referenced temp table.

In [29]:
from assets.mapping import colmap
from utils.db import dataframe_to_db
from di import simple_di

dataframe_to_db(
    simple_di(raw), 
    tablename=source_tablename + '_di'
)

'ushe_students_di'